# Testing ChatESG

In [64]:
import requests
import json
import pandas as pd
import ast
from tqdm import tqdm

In [65]:
TEST_UID = "#test-uid"
MODE = "local"

if MODE=="local":
    base_url = 'http://localhost:8000'

# Health check
requests.get(f"{base_url}/health").content, base_url

(b'{"status":"OK"}', 'http://localhost:8000')

### Chat

In [29]:
headers_chat = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

chat_payload = {
    "src": "skripsi",
    "model": "gpt-4o-mini",
    # "model": 'climategpt-13b',
    # "bsid": "7f79f395c3564549be52",
    "retriever": 'similarity',
    "top_k": 4,
    "inputs": [
        {"type": "text", "role": "user", "content": "sebutkan nama presiden komisaris dari perusahaan"}
    ]
}

response = requests.post(f"{base_url}/api/v1/chat", headers=headers_chat, data=json.dumps(chat_payload), stream=True)
for line in response.iter_lines():
    if line:
        chunk = json.loads(line.decode('utf-8'))
        print("Chunk received:", chunk)
chunk

Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': [], 'sources': None, 'bsid': '2d5e3434380a4733a352', 'bcid': '0fd9f18c-5772-4a52-8fab-5f441c728eb2', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': [''], 'sources': None, 'bsid': '2d5e3434380a4733a352', 'bcid': '0fd9f18c-5772-4a52-8fab-5f441c728eb2', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': ['Nama'], 'sources': None, 'bsid': '2d5e3434380a4733a352', 'bcid': '0fd9f18c-5772-4a52-8fab-5f441c728eb2', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'], 'responses': ['Nama pres'], 'sources': None, 'bsid': '2d5e3434380a4733a352', 'bcid': '0fd9f18c-5772-4a52-8fab-5f441c728eb2', 'metadata': None}
Chunk received: {'done': False, 'tools': ['Mengam

{'done': True,
 'tools': ['Mengambil informasi relevan dari laporan keberlanjutan'],
 'responses': ['Nama presiden komisaris dari perusahaan adalah Prijono Sugiarto.'],
 'sources': [{'url': 'https://www.astra.co.id/',
   'pages': [60, 30, 61, 59],
   'snippet': None,
   'related_indicators': None}],
 'bsid': '2d5e3434380a4733a352',
 'bcid': '0fd9f18c-5772-4a52-8fab-5f441c728eb2',
 'metadata': None}

### Retrieval

In [70]:
headers = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

payload = {
    "query": "Berapa jumlah karyawan perusahaan berdasarkan gender?",
    "filter": {"company": "PT Astra International Tbk", "year": 2022},
    "mode": "indicator-cls",
    "top_k": 4,
    "model": "gpt-4o-mini"
}

response = requests.post(f"{base_url}/api/v1/retrieval", headers=headers, data=json.dumps(payload))
json.loads(response.content.decode('utf-8'))

{'contents': [{'id': None,
   'metadata': {'company': 'PT Astra International Tbk',
    'disclosed_gri': ['405-1'],
    'page': 114.0,
    'ticker': 'ASII',
    'year': 2022.0},
   'page_content': "## Woman Leaders Program\n\nSepanjang tahun 2023, Astra melaksanakan sejumlah aktivitas untuk memperkuat keberagaman dan inklusi yang sejalan dengan Astra 2030 Sustainability Aspirations, seperti:\n\nThroughout 2023, Astra carried out numerous acitivities to strengthen diversity and inclusion in line with the Astra 2030 Sustainability Aspirations, namely:\n\n## DIVERSITY & INCLUSION FOR LEADERS\n\n## IGNITE THE IMPACT OF WOMEN LEADERS\n\n23 batch dengan total 605 peserta manajemen eksekutif dari 94 perusahaan Grup Astra\n\n23 batch with 605 participants from 94 companies across Astra Group\n\n## ASTRA WOMEN SUPPORT SYSTEM\n\n22 sesi mentoring dengan 11 eksekutif wanita dari 11 perusahaan\n\n22 mentoring sessions covering 11 women executives from 11 companies\n\nBRANDING & EDUCATIONAL CONTENT

### Generation Evaluation

In [6]:
response_text="""
Berdasarkan laporan keberlanjutan PT Astra International Tbk tahun 2022, berikut adalah asosiasi industri, asosiasi keanggotaan lain, serta organisasi advokasi nasional atau internasional yang diikuti oleh Astra, di mana organisasi tersebut memiliki peran penting:

### Asosiasi Industri:
1. **Asosiasi Industri Sepeda Motor Indonesia (AISI)**
2. **Gabungan Industri Alat Mobil dan Motor (GIAMM)**
3. **Gabungan Industri Kendaraan Bermotor Indonesia (GAIKINDO)**
4. **Asosiasi Asuransi Jiwa Indonesia (AAJI)**
5. **Asosiasi Asuransi Syariah Indonesia (AASI)**
6. **Asosiasi Asuransi Umum Indonesia (AAUI)**
7. **Asosiasi Jasa Konstruksi Nasional (GAPENSI)**
8. **Asosiasi Kontraktor Indonesia (AKI)**
9. **Perhimpunan Agen Alat Berat Indonesia (PAABI)**
10. **Gabungan Perusahaan Kelapa Sawit Indonesia (GAPKI)**
11. **Himpunan Masyarakat Gambut Indonesia (HGI)**
12. **Asosiasi Jalan Tol Indonesia (ATI)**
13. **Asosiasi Logistik & Forwarder Indonesia (ALFI)**
14. **Asosiasi Perusahaan Rental Kendaraan Indonesia (ASPERKINDO)**
15. **Indonesian National Shipowners' Association (INSA)**
16. **Persatuan Balai Lelang Indonesia (PERBALI)**
17. **Asosiasi Industri Teknologi Informasi (AITI)**
18. **Asosiasi Pengusaha Komputer Indonesia (APKOMINDO)**
19. **Asosiasi Emiten Indonesia (AEI)**
20. **Asosiasi Pengusaha Indonesia (APINDO)**
21. **Kamar Dagang dan Industri Indonesia (KADIN)**

### Organisasi Advokasi dan Konsorsium:
1. **Forum Kemitraan Pembangunan Sosial Suku Anak Dalam (FKPSAD)**
2. **Konsorsium Genom Seluruh Indonesia**
3. **Palm Oil Genome Oil Project**

### Keanggotaan Lain:
1. **Asosiasi Perusahaan Perdagangan Barang Distributor, Keagenan dan Industri Indonesia (ARDIN INDONESIA)**
2. **Asosiasi Perusahaan Pengadaan Komputer dan Telematika Indonesia (ASPEKMI)**
3. **Ikatan Akuntan Indonesia (IAI)**
4. **Indonesia Corporate Secretary Association (ICSA)**

Keterlibatan Astra dalam berbagai asosiasi ini menunjukkan komitmennya untuk berkolaborasi dan berkontribusi dalam pengembangan industri serta advokasi isu-isu yang relevan di tingkat nasional dan internasional.
"""
reference_text="""
PT Astra International aktif terlibat dalam berbagai asosiasi industri dan organisasi, antara lain:

1. **Asosiasi Industri:**
   - Asosiasi Industri Sepeda Motor Indonesia (AISI)
   - Gabungan Industri Alat Mobil dan Motor (GIAMM)
   - Gabungan Industri Kendaraan Bermotor Indonesia (GAIKINDO)
   - Gabungan Perusahaan Kelapa Sawit Indonesia (GAPKI)
   - Asosiasi Jasa Konstruksi Nasional (GAPENSI)

2. **Asosiasi Keanggotaan Lain:**
   - Asosiasi Asuransi Jiwa Indonesia (AAJI)
   - Asosiasi Asuransi Syariah Indonesia (AASI)
   - Asosiasi Emiten Indonesia (AEI)
   - Kamar Dagang dan Industri Indonesia (KADIN)

3. **Organisasi Advokasi:**
   - Forum Kemitraan Pembangunan Sosial Suku Anak Dalam (FKPSAD) / Orang Rimba Social Development Partnership Forum
   - Konsorsium Genom Seluruh Indonesia
   - Palm Oil Genome Oil Project

Keterlibatan dalam organisasi-organisasi ini menunjukkan peran aktif Astra dalam membangun jaringan dan berkontribusi pada perkembangan industri serta advokasi sosial.
"""

In [7]:
headers = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

payload = {
    "response": response_text,
    "reference": reference_text,
}

response = requests.post(f"{base_url}/api/v1/eval/fc_metric", headers=headers, data=json.dumps(payload))
res_fc = json.loads(response.content.decode('utf-8'))
res_fc

In [16]:
headers = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

payload = {
    "response": response_text,
    "reference": reference_text,
    "measure_type": "precision"
}

response = requests.post(f"{base_url}/api/v1/eval/rouge_metric", headers=headers, data=json.dumps(payload))
res_rouge = json.loads(response.content.decode('utf-8'))

res_rouge

{'score': 0.308}

### Retrieval Evaluation

In [61]:
headers = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

payload = {
    "retrieved_contexts": [],
    "reference_contexts": [],
}

response = requests.post(f"{base_url}/api/v1/eval/precision_metric", headers=headers, data=json.dumps(payload))
precision = json.loads(response.content.decode('utf-8'))

response = requests.post(f"{base_url}/api/v1/eval/recall_metric", headers=headers, data=json.dumps(payload))
recall = json.loads(response.content.decode('utf-8'))

precision, recall

({'score': 1.0}, {'score': 1.0})

### Batch Evaluation

In [85]:
data = pd.read_excel("data/Test Data v2.xlsx")
data['Pages'] = data['Pages'].apply(lambda x: ast.literal_eval(x))

data = data.sample(30)

headers = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

In [86]:
import json
import logging
from typing import List, Dict
import pandas as pd
import requests
from tqdm import tqdm

# Konfigurasi Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Definisi base_url, TEST_UID, dan headers
base_url = 'http://localhost:8000'
TEST_UID = "#test-uid"
headers = {
    "Content-Type": "application/json",
    "test-uid": TEST_UID
}

# Fungsi untuk melakukan permintaan retrieval
def retrieve_contexts(session: requests.Session, mode: str, payload: Dict) -> List[int]:
    try:
        response = session.post(f"{base_url}/api/v1/retrieval", headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        return [int(res['metadata']['page']) for res in result.get('contents', [])]
    except requests.RequestException as e:
        logging.error(f"Request failed for mode {mode}: {e}")
        return []
    except (KeyError, ValueError) as e:
        logging.error(f"Error parsing response for mode {mode}: {e}")
        return []

# Fungsi untuk menghitung precision dan recall
def calculate_metrics(session: requests.Session, reference_contexts: List[int], retrieved_contexts: List[int]) -> Dict[str, float]:
    payload = {
        "retrieved_contexts": retrieved_contexts,
        "reference_contexts": reference_contexts,
    }
    metrics = {}
    try:
        # Hitung Precision
        precision_response = session.post(
            f"{base_url}/api/v1/eval/precision_metric",
            headers=headers,
            json=payload
        )
        precision_response.raise_for_status()
        precision = precision_response.json().get('score', 0.0)
        metrics['precision'] = precision

        # Hitung Recall
        recall_response = session.post(
            f"{base_url}/api/v1/eval/recall_metric",
            headers=headers,
            json=payload
        )
        recall_response.raise_for_status()
        recall = recall_response.json().get('score', 0.0)
        metrics['recall'] = recall

    except requests.RequestException as e:
        logging.error(f"Metrics calculation failed: {e}")
        metrics['precision'] = 0.0
        metrics['recall'] = 0.0
    except (KeyError, ValueError) as e:
        logging.error(f"Error parsing metrics response: {e}")
        metrics['precision'] = 0.0
        metrics['recall'] = 0.0

    return metrics

# Fungsi utama untuk memproses data
def process_data(data: pd.DataFrame) -> pd.DataFrame:
    # Inisialisasi kolom baru di DataFrame
    data['retrieved_similarity'] = [[] for _ in range(len(data))]
    data['precision_similarity'] = 0.0
    data['recall_similarity'] = 0.0

    data['retrieved_indicator_cls'] = [[] for _ in range(len(data))]
    data['precision_indicator_cls'] = 0.0
    data['recall_indicator_cls'] = 0.0

    data['retrieved_combination'] = [[] for _ in range(len(data))]
    data['precision_combination'] = 0.0
    data['recall_combination'] = 0.0

    with requests.Session() as session:
        for idx, row in tqdm(data.iterrows(), total=len(data), desc="Processing Rows"):
            question = row['Question']
            reference_pages = row['Pages']  # Asumsikan ini adalah list atau iterable

            # Definisikan payload untuk setiap mode
            payload_similarity = {
                "query": question,
                "filter": {"company": "PT Astra International Tbk", "year": 2022},
                "mode": "similarity",
                "top_k": 4
            }

            payload_indicator_cls = {
                "query": question,
                "filter": {"company": "PT Astra International Tbk", "year": 2022},
                "mode": "indicator-cls",
                "model": "gpt-4o-mini"
            }

            payload_combination = {
                "query": question,
                "filter": {"company": "PT Astra International Tbk", "year": 2022},
                "mode": "combination",
                "top_k": 4,
                "model": "gpt-4o-mini"
            }

            # Ambil konteks untuk setiap mode
            sim = retrieve_contexts(session, "similarity", payload_similarity)
            ind = retrieve_contexts(session, "indicator-cls", payload_indicator_cls)
            com = retrieve_contexts(session, "combination", payload_combination)

            # Simpan retrieved_contexts ke DataFrame
            data.at[idx, 'retrieved_similarity'] = sim
            data.at[idx, 'retrieved_indicator_cls'] = ind
            data.at[idx, 'retrieved_combination'] = com

            # Hitung metrics untuk setiap metode dan simpan ke DataFrame
            for method, retrieved in zip(['similarity', 'indicator_cls', 'combination'], [sim, ind, com]):
                metrics = calculate_metrics(session, reference_pages, retrieved)
                if method == 'similarity':
                    data.at[idx, 'precision_similarity'] = metrics['precision']
                    data.at[idx, 'recall_similarity'] = metrics['recall']
                elif method == 'indicator_cls':
                    data.at[idx, 'precision_indicator_cls'] = metrics['precision']
                    data.at[idx, 'recall_indicator_cls'] = metrics['recall']
                elif method == 'combination':
                    data.at[idx, 'precision_combination'] = metrics['precision']
                    data.at[idx, 'recall_combination'] = metrics['recall']

    return data

# Fungsi untuk menyimpan hasil metrics
def save_metrics(data: pd.DataFrame, filepath: str):
    try:
        data.to_csv(filepath, index=False)
        logging.info(f"Metrics successfully saved to {filepath}")
    except Exception as e:
        logging.error(f"Failed to save metrics to {filepath}: {e}")

In [87]:
# Proses data
processed_data = process_data(data)

# # Tampilkan DataFrame yang telah diproses
# print(processed_data)

# # Simpan DataFrame ke CSV
# save_metrics(processed_data, "processed_data_with_metrics.csv")

Processing Rows: 100%|██████████| 30/30 [04:37<00:00,  9.25s/it]


In [88]:
processed_data.describe()

,precision_similarity,recall_similarity,precision_indicator_cls,recall_indicator_cls,precision_combination,recall_combination
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.091667,0.283333,0.553333,0.750000,0.121111,0.483333
std,0.139013,0.429180,0.371762,0.430517,0.136242,0.499713
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.500000,0.625000,0.000000,0.000000
50%,0.000000,0.000000,0.500000,1.000000,0.083333,0.250000
75%,0.250000,0.500000,1.000000,1.000000,0.200000,1.000000
max,0.500000,1.000000,1.000000,1.000000,0.400000,1.000000
